In [ ]:
import datatable as dt
import pandas as pd
import os
import seaborn as sns

In [ ]:
rename = {}
idx = 0
for i in os.listdir('.'):
    if i[:3]=='GSM' and i[-3:]=='bed' and i.find('Blood')!=-1:
        rename[idx] = {}
        rename[idx]['ID'] = i.split('_')[0]
        rename[idx]["type"] = i.split('Blood-')[1].split('-')[0]
        idx+=1

In [ ]:
type = {}
idx = 0
for i in pd.DataFrame(rename).T['type'].unique():
    type[i] = idx
    idx +=1 
type

In [ ]:
df = pd.read_table("./Blood.metilene.chr2x.noNA.input", header=None)
df.columns = 'chrom	pos'.split('\t')+list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])
df.to_csv('./Blood.metilene.chr2x.noNA.input',sep='\t',index=False, float_format='%.3f')
df

In [ ]:
DMRs = pd.read_table('../metilene_output/Blood.metilene.chr2x.noNA.output', skiprows=90)
DMRs = DMRs.loc[DMRs['chr'] =='chr2']
inDMRs = {}
inDMRs2 = {}
for i in DMRs.index:
    for j in range(int(DMRs['start'][i])+1, int(DMRs['stop'][i]+1)):
        inDMRs[j] = DMRs['sig.comparison'][i]
        inDMRs2[j] = DMRs['p'][i]
DMRs

In [ ]:
DMRs[DMRs['sig.comparison']==57].sort_values('start')

In [ ]:
pd.crosstab(DMRs['sig.comparison'],DMRs['sig.comparison']).sum().sort_values(ascending=0).index[:5]

In [ ]:
id_convert = {}
Predictions_anno = pd.read_table("../metilene_output/Blood.metilene.chr2x.noNA.output", encoding='iso-8859-1', sep='@', header=None).head(90)
for i in Predictions_anno.index:
    id_convert[int(Predictions_anno.loc[i][0].split(':')[0].split('Combination ')[1])] = \
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[0]+'|'+\
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[1][:-1]

In [ ]:
df.index = df['pos']
df = df.loc[df['chrom']=='chr2']

df['inDMR'] = df['pos'].map(inDMRs)
df['p'] = df['pos'].map(inDMRs2)

In [ ]:
for i in pd.crosstab(DMRs['sig.comparison'],DMRs['sig.comparison']).sum().sort_values(ascending=0).index[:5]:
    print(id_convert[i])
    cm = sns.clustermap(df.loc[df['inDMR']==i].sort_values('p')\
                   [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])],\
                   cmap='coolwarm', figsize=[12,3], method='ward', row_cluster=False)
    cm.cax.set_visible(False)
    cm.fig.suptitle(id_convert[i]) 

In [ ]:
for i in [57]:
    print(id_convert[i])
    cm = sns.clustermap(df.loc[df['pos'].map(inDMRs)==i]\
                   [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])],\
                   cmap='coolwarm', figsize=[12,12], method='ward', row_cluster=False)
    cm.cax.set_visible(False)
    cm.fig.suptitle(id_convert[i]) 

In [ ]:
# 3261322	3261610
cm = sns.clustermap(df.loc[(df['pos']>=3259478)&(df['pos']<=3261895)]\
               [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])],\
               cmap='coolwarm', figsize=[12,6], method='ward', row_cluster=False)
cm.cax.set_visible(False)
cm.fig.suptitle('pub') 

In [ ]:
# 3261322	3261610
cm = sns.clustermap(df.loc[(df['pos']>=3261322)&(df['pos']<=3261610)]\
               [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])],\
               cmap='coolwarm', figsize=[12,6], method='ward', row_cluster=False)
cm.cax.set_visible(False)
cm.fig.suptitle('pub') 

In [ ]:
df.loc[df['pos'].map(inDMRs)==i].sort_values('p')